# 3D Object Scanner and Reconstructor
## Unit 4 - Vision and Motion Project

This notebook implements:
1. **Shape from Shading (SfS)** - Single image 3D reconstruction
2. **Photometric Stereo** - Multi-light 3D reconstruction
3. **3D Visualization** - Interactive 3D model viewing
4. **Comparison** - Analyzing different methods

## 1. Installation and Setup

In [ ]:
# Install required packages
!pip install opencv-python-headless numpy matplotlib scipy plotly scikit-image open3d

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import cv2
from scipy import ndimage
from scipy.integrate import cumtrapz
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from google.colab import files
import io
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

print("✓ All packages installed successfully!")

## 2. Utility Functions

In [ ]:
def normalize_image(img):
    """Normalize image to [0, 1] range"""
    return (img - img.min()) / (img.max() - img.min() + 1e-8)

def load_and_preprocess(image_path):
    """Load image and convert to grayscale normalized format"""
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not load image: {image_path}")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return normalize_image(gray.astype(np.float64))

def create_synthetic_sphere(size=256, radius=100):
    """Create a synthetic sphere image for testing"""
    center = size // 2
    y, x = np.ogrid[:size, :size]
    
    # Distance from center
    dist = np.sqrt((x - center)**2 + (y - center)**2)
    
    # Create sphere mask
    mask = dist <= radius
    
    # Calculate z-coordinate (height)
    z = np.zeros((size, size))
    z[mask] = np.sqrt(radius**2 - dist[mask]**2)
    
    # Light direction (from top-left)
    light_dir = np.array([-1, -1, 1])
    light_dir = light_dir / np.linalg.norm(light_dir)
    
    # Calculate surface normals
    grad_x, grad_y = np.gradient(z)
    
    # Create normal vectors
    normals = np.zeros((size, size, 3))
    normals[:, :, 0] = -grad_x
    normals[:, :, 1] = -grad_y
    normals[:, :, 2] = 1
    
    # Normalize
    norm_magnitude = np.sqrt(np.sum(normals**2, axis=2, keepdims=True))
    normals = normals / (norm_magnitude + 1e-8)
    
    # Lambertian shading
    intensity = np.sum(normals * light_dir, axis=2)
    intensity = np.maximum(intensity, 0)
    intensity[~mask] = 0
    
    return normalize_image(intensity), z, mask

print("✓ Utility functions defined")

## 3. Shape from Shading (SfS) Implementation

### Theory:
- Uses brightness variations in a single image to infer 3D shape
- Based on Lambertian reflectance model: I = ρ * (N · L)
- Where N is surface normal and L is light direction

In [ ]:
class ShapeFromShading:
    def __init__(self, image, light_direction=None):
        """
        Initialize Shape from Shading
        
        Parameters:
        -----------
        image : numpy array
            Grayscale image normalized to [0, 1]
        light_direction : numpy array
            3D vector indicating light direction [x, y, z]
        """
        self.image = image
        self.height, self.width = image.shape
        
        # Default light direction (from top-left-front)
        if light_direction is None:
            light_direction = np.array([-1, -1, 1])
        
        self.light_dir = light_direction / np.linalg.norm(light_direction)
        
    def compute_surface_normals(self):
        """
        Compute surface normals from image gradients
        Using gradient space representation (p, q)
        """
        # Compute image gradients
        grad_y, grad_x = np.gradient(self.image)
        
        # For Lambertian surfaces: I = albedo * (N · L)
        # Approximate surface gradient (p, q) from image gradients
        
        # Gradient space: p = dz/dx, q = dz/dy
        # Surface normal: N = [-p, -q, 1] / sqrt(p^2 + q^2 + 1)
        
        # Estimate gradients from intensity (simplified approach)
        p = -grad_x / (self.image + 1e-8)
        q = -grad_y / (self.image + 1e-8)
        
        # Smooth gradients
        p = ndimage.gaussian_filter(p, sigma=2)
        q = ndimage.gaussian_filter(q, sigma=2)
        
        # Construct normal vectors
        normals = np.zeros((self.height, self.width, 3))
        normals[:, :, 0] = -p
        normals[:, :, 1] = -q
        normals[:, :, 2] = 1
        
        # Normalize
        norm_magnitude = np.sqrt(np.sum(normals**2, axis=2, keepdims=True))
        normals = normals / (norm_magnitude + 1e-8)
        
        return normals, p, q
    
    def integrate_surface(self, p, q):
        """
        Integrate surface gradients to recover depth
        Uses line integration method
        """
        # Initialize depth map
        depth = np.zeros((self.height, self.width))
        
        # Integrate along rows (using p = dz/dx)
        for i in range(self.height):
            depth[i, :] = np.cumsum(p[i, :])
        
        # Integrate along columns and average (using q = dz/dy)
        depth_y = np.zeros((self.height, self.width))
        for j in range(self.width):
            depth_y[:, j] = np.cumsum(q[:, j])
        
        # Average both integrations
        depth = (depth + depth_y) / 2
        
        return depth
    
    def reconstruct(self):
        """
        Full reconstruction pipeline
        """
        print("Computing surface normals...")
        normals, p, q = self.compute_surface_normals()
        
        print("Integrating surface...")
        depth = self.integrate_surface(p, q)
        
        # Normalize depth
        depth = normalize_image(depth)
        
        return depth, normals

print("✓ Shape from Shading class defined")

## 4. Photometric Stereo Implementation

### Theory:
- Uses multiple images under different lighting conditions
- More accurate than single-image SfS
- Solves: I = ρ * N · L for multiple light sources

In [ ]:
class PhotometricStereo:
    def __init__(self, images, light_directions):
        """
        Initialize Photometric Stereo
        
        Parameters:
        -----------
        images : list of numpy arrays
            List of images under different lighting
        light_directions : list of numpy arrays
            List of 3D light direction vectors
        """
        self.images = [normalize_image(img) for img in images]
        self.n_images = len(images)
        self.height, self.width = images[0].shape
        
        # Normalize light directions
        self.light_dirs = np.array([l / np.linalg.norm(l) for l in light_directions])
        
        if len(self.images) != len(self.light_dirs):
            raise ValueError("Number of images must match number of light directions")
    
    def compute_normals_and_albedo(self):
        """
        Compute surface normals and albedo using least squares
        
        For each pixel: I = ρ * N · L
        With multiple lights: [I1, I2, ..., In]^T = ρ * [L1, L2, ..., Ln] * N
        Solve: I = L * (ρ * N) using pseudo-inverse
        """
        print("Computing normals using photometric stereo...")
        
        # Stack images into intensity matrix (n_images x height x width)
        I = np.array(self.images)
        
        # Reshape to (height*width x n_images)
        I_reshaped = I.reshape(self.n_images, -1).T
        
        # Light matrix (n_images x 3)
        L = self.light_dirs
        
        # Solve for (ρ * N) using least squares
        # (ρ * N) = (L^T * L)^-1 * L^T * I
        L_pseudo_inv = np.linalg.pinv(L)
        G = I_reshaped @ L_pseudo_inv.T  # (height*width x 3)
        
        # Compute albedo (magnitude of G)
        albedo = np.linalg.norm(G, axis=1)
        albedo = albedo.reshape(self.height, self.width)
        
        # Compute normals (normalize G)
        normals = G / (np.linalg.norm(G, axis=1, keepdims=True) + 1e-8)
        normals = normals.reshape(self.height, self.width, 3)
        
        return normals, albedo
    
    def compute_depth(self, normals):
        """
        Compute depth from normals using integration
        """
        print("Computing depth from normals...")
        
        # Extract surface gradients from normals
        # N = [-p, -q, 1] / sqrt(p^2 + q^2 + 1)
        # Therefore: p = -Nx/Nz, q = -Ny/Nz
        
        p = -normals[:, :, 0] / (normals[:, :, 2] + 1e-8)
        q = -normals[:, :, 1] / (normals[:, :, 2] + 1e-8)
        
        # Integrate using cumulative sum
        depth = np.zeros((self.height, self.width))
        
        # Integrate along x
        for i in range(self.height):
            depth[i, :] = np.cumsum(p[i, :])
        
        # Integrate along y and average
        depth_y = np.zeros((self.height, self.width))
        for j in range(self.width):
            depth_y[:, j] = np.cumsum(q[:, j])
        
        depth = (depth + depth_y) / 2
        
        # Normalize
        depth = normalize_image(depth)
        
        return depth
    
    def reconstruct(self):
        """
        Full photometric stereo reconstruction
        """
        normals, albedo = self.compute_normals_and_albedo()
        depth = self.compute_depth(normals)
        
        return depth, normals, albedo

print("✓ Photometric Stereo class defined")

## 5. Visualization Functions

In [ ]:
def visualize_normals(normals, title="Surface Normals"):
    """
    Visualize surface normals as RGB image
    """
    # Convert normals to RGB (map [-1,1] to [0,255])
    normal_rgb = ((normals + 1) / 2 * 255).astype(np.uint8)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(normal_rgb)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.colorbar(label='Normal Direction', shrink=0.8)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def visualize_depth_map(depth, title="Depth Map"):
    """
    Visualize depth map as heatmap
    """
    plt.figure(figsize=(10, 8))
    plt.imshow(depth, cmap='viridis')
    plt.title(title, fontsize=14, fontweight='bold')
    plt.colorbar(label='Depth (normalized)', shrink=0.8)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def plot_3d_surface_matplotlib(depth, title="3D Surface Reconstruction"):
    """
    Plot 3D surface using matplotlib
    """
    height, width = depth.shape
    x = np.linspace(0, width-1, width)
    y = np.linspace(0, height-1, height)
    X, Y = np.meshgrid(x, y)
    
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Downsample for better performance
    step = max(1, min(height, width) // 100)
    surf = ax.plot_surface(X[::step, ::step], Y[::step, ::step], depth[::step, ::step],
                          cmap='viridis', alpha=0.9, edgecolor='none')
    
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Depth')
    fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.tight_layout()
    plt.show()

def plot_3d_surface_plotly(depth, title="Interactive 3D Surface"):
    """
    Create interactive 3D plot using Plotly
    """
    height, width = depth.shape
    x = np.linspace(0, width-1, width)
    y = np.linspace(0, height-1, height)
    
    fig = go.Figure(data=[go.Surface(
        z=depth,
        x=x,
        y=y,
        colorscale='Viridis',
        colorbar=dict(title="Depth")
    )])
    
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Depth',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        width=800,
        height=800
    )
    
    fig.show()

def compare_results(original, sfs_depth, ps_depth):
    """
    Compare original image with reconstructions
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    axes[0].imshow(original, cmap='gray')
    axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(sfs_depth, cmap='viridis')
    axes[1].set_title('Shape from Shading', fontsize=12, fontweight='bold')
    axes[1].axis('off')
    
    axes[2].imshow(ps_depth, cmap='viridis')
    axes[2].set_title('Photometric Stereo', fontsize=12, fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

print("✓ Visualization functions defined")

## 6. Demo: Synthetic Sphere Test

Let's test our implementation on a synthetic sphere where we know the ground truth.

In [ ]:
# Create synthetic sphere
print("Creating synthetic sphere...")
sphere_image, true_depth, mask = create_synthetic_sphere(size=256, radius=100)

# Display original
plt.figure(figsize=(8, 6))
plt.imshow(sphere_image, cmap='gray')
plt.title('Synthetic Sphere (Input Image)', fontsize=14, fontweight='bold')
plt.colorbar(shrink=0.8)
plt.axis('off')
plt.tight_layout()
plt.show()

### Test Shape from Shading

In [ ]:
# Apply Shape from Shading
print("\n=== Shape from Shading ===")
sfs = ShapeFromShading(sphere_image, light_direction=np.array([-1, -1, 1]))
sfs_depth, sfs_normals = sfs.reconstruct()

# Visualize results
visualize_depth_map(sfs_depth, "SfS: Reconstructed Depth")
visualize_normals(sfs_normals, "SfS: Surface Normals")
plot_3d_surface_matplotlib(sfs_depth, "SfS: 3D Reconstruction")

### Test Photometric Stereo

In [ ]:
# Create synthetic images under different lighting
print("\n=== Photometric Stereo ===")
print("Generating images under different lights...")

# Define multiple light directions
light_directions = [
    np.array([-1, -1, 1]),   # Top-left
    np.array([1, -1, 1]),    # Top-right
    np.array([0, 1, 1]),     # Bottom
    np.array([0, 0, 1])      # Front
]

# Generate images
ps_images = []
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for i, light_dir in enumerate(light_directions):
    # Normalize light direction
    light = light_dir / np.linalg.norm(light_dir)
    
    # Calculate surface normals (reusing from sphere generation)
    y, x = np.ogrid[:256, :256]
    center = 128
    radius = 100
    dist = np.sqrt((x - center)**2 + (y - center)**2)
    mask = dist <= radius
    
    z = np.zeros((256, 256))
    z[mask] = np.sqrt(radius**2 - dist[mask]**2)
    
    grad_x, grad_y = np.gradient(z)
    normals = np.zeros((256, 256, 3))
    normals[:, :, 0] = -grad_x
    normals[:, :, 1] = -grad_y
    normals[:, :, 2] = 1
    norm_mag = np.sqrt(np.sum(normals**2, axis=2, keepdims=True))
    normals = normals / (norm_mag + 1e-8)
    
    # Lambertian shading
    intensity = np.sum(normals * light, axis=2)
    intensity = np.maximum(intensity, 0)
    intensity[~mask] = 0
    intensity = normalize_image(intensity)
    
    ps_images.append(intensity)
    
    axes[i].imshow(intensity, cmap='gray')
    axes[i].set_title(f'Light {i+1}', fontsize=10)
    axes[i].axis('off')

plt.suptitle('Input Images with Different Lighting', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Apply Photometric Stereo
ps = PhotometricStereo(ps_images, light_directions)
ps_depth, ps_normals, ps_albedo = ps.reconstruct()

# Visualize results
visualize_depth_map(ps_depth, "Photometric Stereo: Reconstructed Depth")
visualize_normals(ps_normals, "Photometric Stereo: Surface Normals")
visualize_depth_map(ps_albedo, "Photometric Stereo: Albedo Map")
plot_3d_surface_matplotlib(ps_depth, "Photometric Stereo: 3D Reconstruction")

### Compare Methods

In [ ]:
compare_results(sphere_image, sfs_depth, ps_depth)

# Interactive 3D comparison
print("\nInteractive 3D Visualizations:")
plot_3d_surface_plotly(sfs_depth, "Shape from Shading - Interactive")
plot_3d_surface_plotly(ps_depth, "Photometric Stereo - Interactive")

## 7. Upload Your Own Images

Now you can upload your own images to test the reconstruction!

In [ ]:
# Upload single image for Shape from Shading
print("Upload an image for Shape from Shading:")
uploaded = files.upload()

if uploaded:
    # Get the first uploaded file
    filename = list(uploaded.keys())[0]
    
    # Load image
    img_data = uploaded[filename]
    img = Image.open(io.BytesIO(img_data))
    img_array = np.array(img.convert('L'))  # Convert to grayscale
    img_normalized = normalize_image(img_array.astype(np.float64))
    
    # Display original
    plt.figure(figsize=(8, 6))
    plt.imshow(img_normalized, cmap='gray')
    plt.title('Uploaded Image', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Apply Shape from Shading
    print("\nProcessing with Shape from Shading...")
    sfs_user = ShapeFromShading(img_normalized)
    user_depth, user_normals = sfs_user.reconstruct()
    
    # Visualize
    visualize_depth_map(user_depth, "Your Image: Depth Map")
    visualize_normals(user_normals, "Your Image: Surface Normals")
    plot_3d_surface_matplotlib(user_depth, "Your Image: 3D Reconstruction")
    plot_3d_surface_plotly(user_depth, "Your Image: Interactive 3D")

### Upload Multiple Images for Photometric Stereo

In [ ]:
# Upload multiple images (same object, different lighting)
print("Upload 3-4 images of the SAME object under DIFFERENT lighting:")
uploaded_multi = files.upload()

if len(uploaded_multi) >= 3:
    # Load all images
    user_images = []
    filenames = list(uploaded_multi.keys())
    
    fig, axes = plt.subplots(1, len(filenames), figsize=(4*len(filenames), 4))
    if len(filenames) == 1:
        axes = [axes]
    
    for i, filename in enumerate(filenames):
        img_data = uploaded_multi[filename]
        img = Image.open(io.BytesIO(img_data))
        img_array = np.array(img.convert('L'))
        img_normalized = normalize_image(img_array.astype(np.float64))
        user_images.append(img_normalized)
        
        axes[i].imshow(img_normalized, cmap='gray')
        axes[i].set_title(f'Image {i+1}', fontsize=10)
        axes[i].axis('off')
    
    plt.suptitle('Uploaded Images', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Define light directions (you may need to adjust these based on your setup)
    n_images = len(user_images)
    angles = np.linspace(0, 2*np.pi, n_images, endpoint=False)
    user_light_dirs = []
    for angle in angles:
        light = np.array([np.cos(angle), np.sin(angle), 1])
        user_light_dirs.append(light)
    
    print(f"\nProcessing {n_images} images with Photometric Stereo...")
    print("Note: Light directions are estimated. For best results, specify actual light positions.")
    
    # Apply Photometric Stereo
    ps_user = PhotometricStereo(user_images, user_light_dirs)
    user_ps_depth, user_ps_normals, user_ps_albedo = ps_user.reconstruct()
    
    # Visualize
    visualize_depth_map(user_ps_depth, "Your Images: Depth Map")
    visualize_normals(user_ps_normals, "Your Images: Surface Normals")
    visualize_depth_map(user_ps_albedo, "Your Images: Albedo")
    plot_3d_surface_matplotlib(user_ps_depth, "Your Images: 3D Reconstruction")
    plot_3d_surface_plotly(user_ps_depth, "Your Images: Interactive 3D")
else:
    print("Please upload at least 3 images for Photometric Stereo!")

## 8. Advanced: Creating Test Dataset

Generate multiple synthetic objects for testing

In [ ]:
def create_synthetic_objects():
    """Create various synthetic 3D objects for testing"""
    
    size = 256
    objects = {}
    
    # 1. Sphere (already have this)
    sphere_img, sphere_depth, _ = create_synthetic_sphere(size, 100)
    objects['Sphere'] = (sphere_img, sphere_depth)
    
    # 2. Cylinder
    y, x = np.ogrid[:size, :size]
    center_x = size // 2
    dist_x = np.abs(x - center_x)
    radius = 80
    
    cylinder_mask = dist_x <= radius
    cylinder_z = np.zeros((size, size))
    cylinder_z[cylinder_mask] = np.sqrt(radius**2 - dist_x[cylinder_mask]**2)
    
    # Render with lighting
    light = np.array([-1, -1, 1]) / np.linalg.norm([-1, -1, 1])
    grad_x, grad_y = np.gradient(cylinder_z)
    normals = np.zeros((size, size, 3))
    normals[:, :, 0] = -grad_x
    normals[:, :, 1] = -grad_y
    normals[:, :, 2] = 1
    norm_mag = np.sqrt(np.sum(normals**2, axis=2, keepdims=True))
    normals = normals / (norm_mag + 1e-8)
    
    cylinder_img = np.maximum(np.sum(normals * light, axis=2), 0)
    cylinder_img[~cylinder_mask] = 0
    objects['Cylinder'] = (normalize_image(cylinder_img), cylinder_z)
    
    # 3. Cone
    y, x = np.ogrid[:size, :size]
    center = size // 2
    dist = np.sqrt((x - center)**2 + (y - center)**2)
    max_radius = 100
    
    cone_mask = dist <= max_radius
    cone_z = np.zeros((size, size))
    cone_z[cone_mask] = max_radius - dist[cone_mask]
    
    # Render
    grad_x, grad_y = np.gradient(cone_z)
    normals = np.zeros((size, size, 3))
    normals[:, :, 0] = -grad_x
    normals[:, :, 1] = -grad_y
    normals[:, :, 2] = 1
    norm_mag = np.sqrt(np.sum(normals**2, axis=2, keepdims=True))
    normals = normals / (norm_mag + 1e-8)
    
    cone_img = np.maximum(np.sum(normals * light, axis=2), 0)
    cone_img[~cone_mask] = 0
    objects['Cone'] = (normalize_image(cone_img), cone_z)
    
    return objects

# Generate and visualize synthetic objects
print("Generating synthetic test objects...")
test_objects = create_synthetic_objects()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (name, (img, depth)) in enumerate(test_objects.items()):
    axes[idx*2].imshow(img, cmap='gray')
    axes[idx*2].set_title(f'{name} - Image', fontsize=12, fontweight='bold')
    axes[idx*2].axis('off')
    
    axes[idx*2+1].imshow(depth, cmap='viridis')
    axes[idx*2+1].set_title(f'{name} - True Depth', fontsize=12, fontweight='bold')
    axes[idx*2+1].axis('off')

plt.tight_layout()
plt.show()

print("\nYou can now test reconstruction on these objects!")

## 9. Performance Analysis

Compare reconstruction quality quantitatively

In [ ]:
def compute_reconstruction_error(true_depth, reconstructed_depth, mask=None):
    """
    Compute error metrics between true and reconstructed depth
    """
    if mask is None:
        mask = np.ones_like(true_depth, dtype=bool)
    
    # Normalize both to same scale
    true_norm = normalize_image(true_depth[mask])
    recon_norm = normalize_image(reconstructed_depth[mask])
    
    # Mean Absolute Error
    mae = np.mean(np.abs(true_norm - recon_norm))
    
    # Root Mean Square Error
    rmse = np.sqrt(np.mean((true_norm - recon_norm)**2))
    
    # Peak Signal-to-Noise Ratio
    mse = np.mean((true_norm - recon_norm)**2)
    psnr = 10 * np.log10(1.0 / (mse + 1e-10))
    
    return {'MAE': mae, 'RMSE': rmse, 'PSNR': psnr}

# Analyze sphere reconstruction
print("\n=== Reconstruction Quality Analysis ===")
print("\nSphere Reconstruction:")

sfs_errors = compute_reconstruction_error(true_depth, sfs_depth, mask)
ps_errors = compute_reconstruction_error(true_depth, ps_depth, mask)

print("\nShape from Shading:")
for metric, value in sfs_errors.items():
    print(f"  {metric}: {value:.4f}")

print("\nPhotometric Stereo:")
for metric, value in ps_errors.items():
    print(f"  {metric}: {value:.4f}")

# Visualization of error
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sfs_error = np.abs(normalize_image(true_depth) - normalize_image(sfs_depth))
ps_error = np.abs(normalize_image(true_depth) - normalize_image(ps_depth))

im1 = axes[0].imshow(sfs_error, cmap='hot')
axes[0].set_title('SfS Error Map', fontsize=12, fontweight='bold')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], shrink=0.8)

im2 = axes[1].imshow(ps_error, cmap='hot')
axes[1].set_title('Photometric Stereo Error Map', fontsize=12, fontweight='bold')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1], shrink=0.8)

plt.tight_layout()
plt.show()

## 10. Export Results

Save reconstructed 3D models

In [ ]:
def export_depth_as_obj(depth, filename='reconstruction.obj', scale=1.0):
    """
    Export depth map as OBJ file (3D mesh format)
    """
    height, width = depth.shape
    
    with open(filename, 'w') as f:
        # Write vertices
        for i in range(height):
            for j in range(width):
                x = j * scale
                y = i * scale
                z = depth[i, j] * scale * 100  # Scale z for visibility
                f.write(f"v {x} {y} {z}\n")
        
        # Write faces (triangles)
        for i in range(height - 1):
            for j in range(width - 1):
                # Vertex indices (1-indexed)
                v1 = i * width + j + 1
                v2 = i * width + j + 2
                v3 = (i + 1) * width + j + 1
                v4 = (i + 1) * width + j + 2
                
                # Two triangles per quad
                f.write(f"f {v1} {v2} {v3}\n")
                f.write(f"f {v2} {v4} {v3}\n")
    
    print(f"✓ Exported to {filename}")
    return filename

# Export reconstructions
print("Exporting 3D models...")
sfs_file = export_depth_as_obj(sfs_depth, 'sfs_reconstruction.obj')
ps_file = export_depth_as_obj(ps_depth, 'ps_reconstruction.obj')

print("\nDownload the OBJ files to view in 3D software (Blender, MeshLab, etc.)")
files.download('sfs_reconstruction.obj')
files.download('ps_reconstruction.obj')

## 11. Summary and Conclusions

### Key Findings:

1. **Shape from Shading**:
   - Works with single image
   - Requires known lighting conditions
   - Fast but less accurate
   - Good for quick depth estimation

2. **Photometric Stereo**:
   - Requires multiple images
   - More accurate surface normals
   - Better depth reconstruction
   - Handles complex surfaces better

3. **Practical Applications**:
   - 3D scanning and modeling
   - Face recognition and reconstruction
   - Object inspection and quality control
   - Archaeological artifact documentation
   - Medical imaging

### Future Improvements:
- Implement more robust integration methods
- Add texture mapping
- Handle specular reflections
- Real-time reconstruction
- Deep learning-based enhancement

## 12. Additional Resources

### References:
1. Horn, B. K. P. (1970). "Shape from Shading"
2. Woodham, R. J. (1980). "Photometric Method for Determining Surface Orientation from Multiple Images"
3. Horn & Brooks (1989). "Shape from Shading"

### Suggested Readings:
- Computer Vision: Algorithms and Applications by Richard Szeliski
- Multiple View Geometry in Computer Vision by Hartley & Zisserman
- 3D Computer Vision: Efficient Methods and Applications by Klette

### Tools:
- OpenCV: https://opencv.org/
- Open3D: http://www.open3d.org/
- MeshLab: https://www.meshlab.net/